In [1]:
%matplotlib notebook
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv


import pandas as pd

import pickle
import os.path

import warnings
warnings.simplefilter('ignore')

from scipy.optimize import minimize_scalar

from poly_basis import *

from ex2_helper import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [179]:
'''
def rbf(r):
    return r**9
def l_rbf(r):
    return 9*8*r**7 + 2*9*r**7
'''

#eps = 4
def rbf(r):
    return np.exp(-eps * r**2)
def rbfd_div_r(r):
    return -2*eps*rbf(r)
def rbfd2(r):
    return 2*eps*rbf(r)*(2*eps*r-1)

def rbfL(r):
    return rbfd2(r) + rbfd_div_r(r) *2

def grad_rbf(node, center):
    r = dist(node, center)
    d_phi = -2*eps*rbf(r)
    return node * d_phi


def foo1(node):
    return -node[0]*2
def exact1(node):
    return node[0]

# l=3
def foo2(node):
    return -6 * (node[0]**2 - node[1]**2)
def exact2(node):
    return (node[0]**2 - node[1]**2)

def foo3(node):
    return -(node[0]**2 - node[1]**2) * node[2] * 12

def exact3(node):
    return (node[0]**2 - node[1]**2) * node[2]

# l=4
def foo4(node):
    return -(3*node[0]**2 - node[1]**2)*node[1]*node[2] * 20
def exact4(node):
    return (3*node[0]**2 - node[1]**2)*node[1]*node[2]

In [194]:
n = 400
k = 41
delta = .2
gamma = 1
my_node = 51
eps = 2

foo, exact = foo1, exact1

In [195]:
nodes = gen_spiral_nodes(n)
tree = cKDTree(np.array(nodes))
stencil = tree.query(nodes[my_node], k)[1]
nn = np.array([nodes[i] for i in stencil])

In [196]:
nodes = gen_spiral_nodes(n)
tree = cKDTree(np.array(nodes))
stencil = tree.query(nodes[my_node], k)[1]
nn = np.array([nodes[i] for i in stencil])

dist_mat = [[dist(x,y) for x in nn] for y in nn]
d = np.min(np.diag([10**5]*k) + dist_mat)*delta

nnp = nn*(1+d)
nnm = nn*(1-d)
nna = np.concatenate([[nn],[nnp],[nnm]]).reshape((3*k, 3))

# reconstruct surface
A = np.array([[rbf(dist(x,y)) for x in nna] for y in nna])
#fsa = np.array([exact(x) for x in nn]*3)
f_level = [0.0]*k + [1.0]*k+ [-1.0]*k
f_level = np.array(f_level)
cs = la.solve(A,f_level)

grad_s = np.zeros((k, 3))
for c, x in zip(cs, nna):
    grad_s += [c*grad_rbf(x,y) for y in nn]

normals = np.array([gs/norm(gs) for gs in grad_s])
#for i, (node, normal) in enumerate(zip(nn, normals)):
#    if norm(node - normal) > 1:
#        normals[i] = -normals[i]

#yyp = nn*(1+gamma*d)
#yym = nn*(1-gamma*d)
yyp = nn + normals*gamma*d
yym = nn - normals*gamma*d

yya = np.concatenate([[nn],[yyp],[yym]]).reshape((3*k, 3))

'''
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(nnp[:,0], nnp[:,1], nnp[:,2], '.', color='r')
ax.scatter(yyp[:,0], yyp[:,1], yyp[:,2], '.', color='c')
ax.scatter( nn[:,0],  nn[:,1],  nn[:,2], '.', color='b')
ax.scatter(yym[:,0], yym[:,1], yym[:,2], '.', color='y')
ax.scatter(nnm[:,0], nnm[:,1], nnm[:,2], '.', color='g')
ax.scatter(0, 0, 0, '.', color='k')

plt.show()
'''

A = np.array([[rbf(dist(x,y)) for x in yya] for y in nna])
B = np.array([[rbfL(dist(x,y)) for x in yya] for y in nna])

print('cond A: %g' % cond(A))
print('cond B: %g' % cond(A))

fs = np.array([exact(x) for x in nna])
us = B@la.solve(A, fs)

#D = B * inv(A)
#D = D[:k, :k] + D[:k, k:2*k] + D[:k, 2*k:3*k]


ds = np.array([foo(x) for x in nn])

print(norm(us[:k] - ds)/norm(ds))
print(((us)[0] - foo(nn[0]))/foo(nn[0]))

cond A: 1.92579e+14
cond B: 1.92579e+14
6876.273474267327
22104.898096283


In [87]:
cond: 1.18112e+09
2251585.55225971
2579107.366339111

array([[0.99716724, 0.59058204, 0.73274654],
       [0.59057333, 0.99716724, 0.67010916],
       [0.73274352, 0.67011336, 0.99716724]])

In [42]:
rbf(dist(yya[-2], nna[-1]))

0.4882599028434908

In [50]:
np.max(ds)

1.9502455490380237

In [ ]:
grad_mat

In [ ]:
grad_s

In [ ]:
normals

In [ ]:
yyp + nn

In [ ]:
nn - yyp

In [ ]:
normals